In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import numpy as np

res = 32
boxHeight = 1.0
aspect_ratio = 0.5

# A mesh to solve velocity and pressure

VPmesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q2/dPc1"), 
                                   elementRes  = (int(res * aspect_ratio), res), 
                                   minCoord    = (0., 0.), 
                                   maxCoord    = (boxHeight*aspect_ratio, boxHeight))

velocityField    = uw.mesh.MeshVariable( mesh=VPmesh,         nodeDofCount=2 )
pressureField    = uw.mesh.MeshVariable( mesh=VPmesh.subMesh, nodeDofCount=1 )

velocityField.data[:,:] = 0.0
pressureField.data[:] = 0.0


# Boundary conditions - free slip:
#    Vx on side walls (0) 
#    Vy on top / bottom (0)

vxWalls = VPmesh.specialSets["MinI_VertexSet"] + \
          VPmesh.specialSets["MaxI_VertexSet"] 

vyWalls = VPmesh.specialSets["MinJ_VertexSet"] + \
          VPmesh.specialSets["MaxJ_VertexSet"]

## Instructions !! 

Add an internal velocity contraint to these boundary conditions. One internal velocity
specified in the mesh which shows how the flow is not localised around buoyancy sources or driving
velocities. 

You might choose the middle node of the mesh 

```
node = len(MyMesh.data)/2
```

But you will also need to know the method for adding a specified node to the set of 
indices in the existing collections (`vxWalls` or `vyWalls` ... though you might 
want to rename them now !).

```
vxCondition.add([node1, node2, ...] )
vyCondition.add([node1, node2, ...] )

```

Don't forget to set the corresponding value in the velocity data structure.

## Your wonderful code goes here:

## Test it !

In [ ]:
# Now register this information

velocityBC = uw.conditions.DirichletCondition( variable = velocityField, 
                                               indexSetsPerDof = (vxWalls, vyWalls) )


# The equations are templated already 

stokesPIC = uw.systems.Stokes( velocityField  = velocityField, 
                               pressureField  = pressureField,
                               conditions     = [velocityBC,],
                               fn_viscosity   = 1.0, 
                               fn_bodyforce   = (0.0,0.0) )

# And a suitable solver package is already attached to it

solver = uw.systems.Solver( stokesPIC )

# Solve and see if it worked 

solver.solve()

fig2 = glucifer.Figure()
fig2.append( glucifer.objects.VectorArrows( VPmesh, velocityField, arrowHead=0.2, scaling=0.1 ) )
fig2.append( glucifer.objects.Surface( VPmesh, pressureField ) )
fig2.show()

## Solution

Just in case, this is ~~the~~ a possible [answer](022-Exercise-PointSource-Solution.ipynb)